In [1]:
!pip install tqdm
!pip install dbfread
!pip install pandas
!pip install pandas pyarrow
!pip install xgboost
!pip install pyarrow

In [2]:
from collections import Counter
import datetime
from datetime import date, datetime
import gc
import itertools
import json
import os
import pickle
import re
from sys import prefix

import pandas as pd
from pandas import DataFrame
from pandas.api.types import is_numeric_dtype
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse import hstack
from scipy.sparse import save_npz
from scipy.sparse import load_npz
import pandas as pd

from google.colab import drive

from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, log_loss, classification_report
import xgboost as xgb
from xgboost import XGBClassifier

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
WORKSPACE_PATH = '/content/drive/MyDrive/docs/keepForever/mipt/flactory_dataton/'
WORKSPACE_TMP_PATH = WORKSPACE_PATH + 'tmp/' # внимание, очищается полностью

DF_PATH = WORKSPACE_PATH + 'df/'
META_DF_PATH = DF_PATH + 'meta_df.parquet'
ORDER_DF_PATH = DF_PATH + 'order_df.parquet'
ALL_ORDERS_DF_PATH = DF_PATH + 'df_all_orders.parquet' # для хранения промежуточного набора для обучающей выборки
ALL_ORDERS_VAL_DF_PATH = DF_PATH + 'val_df_all_orders.parquet' # для хранения промежуточного набора для валидационной выборки
ALL_ORDERS_TEST_DF_PATH = DF_PATH + 'test_df_all_orders.parquet' # для хранения промежуточного набора для тестовой выборки

# наборы с готовыми к прогнозированию но невыровненными столбами относительно друг друга
TRAIN_PROCESSED = DF_PATH + 'train_preprocessed_df.pkl'
VAL_PROCESSED = DF_PATH + 'val_preprocessed_df.pkl'
TEST_PROCESSED = DF_PATH + 'test_preprocessed_df.pkl'

Y_TRAIN_DF = DF_PATH + 'x_train_df.parquet'
X_VAL_DF = DF_PATH + 'x_val_df.parquet'
X_TEST_DF = DF_PATH + 'x_test_df.parquet'

TEST_PRED = DF_PATH + 'y_test_pred.csv'

TOP_FEATURES_PATH = DF_PATH + 'top_features.pkl'

INPUT_PATH = WORKSPACE_PATH + 'input/'
TEST_INPUT_PATH = INPUT_PATH + 'test.json'
VAL_INPUT_PATH = INPUT_PATH + 'val.json'
TRAIN_INPUT_PATH = INPUT_PATH + 'train.json'
GEN_CAT_MAPPING_INPUT_PATH = INPUT_PATH + 'general_categories_mapping.json'

# тренировочный набор (определены с запасом, может пригодиться несколько квартетов наборов)
ORDERS_PER_SITE_MATR_PATH_1 = DF_PATH + "orders_per_site_matr_1.npz"
ORDERS_PER_SITE_COLUMNS_PATH_1 = DF_PATH + "orders_per_site_columns_1.csv"
ORDERS_PER_SITE_INDEX_PATH_1 = DF_PATH + "orders_per_site_index_1.csv"

ITEMS_PER_SITE_MATR_PATH_1 = DF_PATH + "items_per_site_matr_1.npz"
ITEMS_PER_SITE_COLUMNS_PATH_1 = DF_PATH + "items_per_site_columns_1.csv"
ITEMS_PER_SITE_INDEX_PATH_1 = DF_PATH + "items_per_site_index_1.csv"

ITEM_PER_BRAND_MATR_PATH_1 = DF_PATH + "item_per_brand_matr_1.npz"
ITEM_PER_BRAND_COLUMNS_PATH_1 = DF_PATH + "item_per_brand_columns_1.csv"
ITEM_PER_BRAND_INDEX_PATH_1 = DF_PATH + "item_per_brand_index_1.csv"

ITEMS_PER_CATEGORY_MATR_PATH_1 = DF_PATH + "items_per_category_matr_1.npz"
ITEMS_PER_CATEGORY_COLUMNS_PATH_1 = DF_PATH + "items_per_category_columns_1.csv"
ITEMS_PER_CATEGORY_INDEX_PATH_1 = DF_PATH + "items_per_category_index_1.csv"

ORDERS_PER_SITE_MATR_PATH_2 = DF_PATH + "orders_per_site_matr_2.npz"
ORDERS_PER_SITE_COLUMNS_PATH_2 = DF_PATH + "orders_per_site_columns_2.csv"
ORDERS_PER_SITE_INDEX_PATH_2 = DF_PATH + "orders_per_site_index_2.csv"

ITEMS_PER_SITE_MATR_PATH_2 = DF_PATH + "items_per_site_matr_2.npz"
ITEMS_PER_SITE_COLUMNS_PATH_2 = DF_PATH + "items_per_site_columns_2.csv"
ITEMS_PER_SITE_INDEX_PATH_2 = DF_PATH + "items_per_site_index_2.csv"

ITEM_PER_BRAND_MATR_PATH_2 = DF_PATH + "item_per_brand_matr_2.npz"
ITEM_PER_BRAND_COLUMNS_PATH_2 = DF_PATH + "item_per_brand_columns_2.csv"
ITEM_PER_BRAND_INDEX_PATH_2 = DF_PATH + "item_per_brand_index_2.csv"

ITEMS_PER_CATEGORY_MATR_PATH_2 = DF_PATH + "items_per_category_matr_2.npz"
ITEMS_PER_CATEGORY_COLUMNS_PATH_2 = DF_PATH + "items_per_category_columns_2.csv"
ITEMS_PER_CATEGORY_INDEX_PATH_2 = DF_PATH + "items_per_category_index_2.csv"

ORDERS_PER_SITE_MATR_PATH_3 = DF_PATH + "orders_per_site_matr_3.npz"
ORDERS_PER_SITE_COLUMNS_PATH_3 = DF_PATH + "orders_per_site_columns_3.csv"
ORDERS_PER_SITE_INDEX_PATH_3 = DF_PATH + "orders_per_site_index_3.csv"

ITEMS_PER_SITE_MATR_PATH_3 = DF_PATH + "items_per_site_matr_3.npz"
ITEMS_PER_SITE_COLUMNS_PATH_3 = DF_PATH + "items_per_site_columns_3.csv"
ITEMS_PER_SITE_INDEX_PATH_3 = DF_PATH + "items_per_site_index_3.csv"

ITEM_PER_BRAND_MATR_PATH_3 = DF_PATH + "item_per_brand_matr_3.npz"
ITEM_PER_BRAND_COLUMNS_PATH_3 = DF_PATH + "item_per_brand_columns_3.csv"
ITEM_PER_BRAND_INDEX_PATH_3 = DF_PATH + "item_per_brand_index_3.csv"

ITEMS_PER_CATEGORY_MATR_PATH_3 = DF_PATH + "items_per_category_matr_3.npz"
ITEMS_PER_CATEGORY_COLUMNS_PATH_3 = DF_PATH + "items_per_category_columns_3.csv"
ITEMS_PER_CATEGORY_INDEX_PATH_3 = DF_PATH + "items_per_category_index_3.csv"

# валидационный набор
ORDERS_PER_SITE_MATR_PATH_VAL = DF_PATH + "orders_per_site_matr_val.npz"
ORDERS_PER_SITE_COLUMNS_PATH_VAL = DF_PATH + "orders_per_site_columns_val.csv"
ORDERS_PER_SITE_INDEX_PATH_VAL = DF_PATH + "orders_per_site_index_val.csv"

ITEMS_PER_SITE_MATR_PATH_VAL = DF_PATH + "items_per_site_matr_val.npz"
ITEMS_PER_SITE_COLUMNS_PATH_VAL = DF_PATH + "items_per_site_columns_val.csv"
ITEMS_PER_SITE_INDEX_PATH_VAL = DF_PATH + "items_per_site_index_val.csv"

ITEM_PER_BRAND_MATR_PATH_VAL = DF_PATH + "item_per_brand_matr_val.npz"
ITEM_PER_BRAND_COLUMNS_PATH_VAL = DF_PATH + "item_per_brand_columns_val.csv"
ITEM_PER_BRAND_INDEX_PATH_VAL = DF_PATH + "item_per_brand_index_val.csv"

ITEMS_PER_CATEGORY_MATR_PATH_VAL = DF_PATH + "items_per_category_matr_val.npz"
ITEMS_PER_CATEGORY_COLUMNS_PATH_VAL = DF_PATH + "items_per_category_columns_val.csv"
ITEMS_PER_CATEGORY_INDEX_PATH_VAL = DF_PATH + "items_per_category_index_val.csv"

# тестовый набор
ORDERS_PER_SITE_MATR_PATH_TEST = DF_PATH + "orders_per_site_matr_test.npz"
ORDERS_PER_SITE_COLUMNS_PATH_TEST = DF_PATH + "orders_per_site_columns_test.csv"
ORDERS_PER_SITE_INDEX_PATH_TEST = DF_PATH + "orders_per_site_index_test.csv"

ITEMS_PER_SITE_MATR_PATH_TEST = DF_PATH + "items_per_site_matr_test.npz"
ITEMS_PER_SITE_COLUMNS_PATH_TEST = DF_PATH + "items_per_site_columns_test.csv"
ITEMS_PER_SITE_INDEX_PATH_TEST = DF_PATH + "items_per_site_index_test.csv"

ITEM_PER_BRAND_MATR_PATH_TEST = DF_PATH + "item_per_brand_matr_test.npz"
ITEM_PER_BRAND_COLUMNS_PATH_TEST = DF_PATH + "item_per_brand_columns_test.csv"
ITEM_PER_BRAND_INDEX_PATH_TEST = DF_PATH + "item_per_brand_index_test.csv"

ITEMS_PER_CATEGORY_MATR_PATH_TEST = DF_PATH + "items_per_category_matr_test.npz"
ITEMS_PER_CATEGORY_COLUMNS_PATH_TEST = DF_PATH + "items_per_category_columns_test.csv"
ITEMS_PER_CATEGORY_INDEX_PATH_TEST = DF_PATH + "items_per_category_index_test.csv"

In [5]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
def save_object_to_file(obj, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(obj, file)
    print(f"Пусть сохранения {file_path}")

def load_object_from_file(file_path):
    with open(file_path, 'rb') as file:
        obj = pickle.load(file)
    return obj

#### Считать обучающий набор

In [ ]:
with open(TRAIN_INPUT_PATH) as file:
    train_file = file.read()

train_json = json.loads(train_file)

In [ ]:
with open(GEN_CAT_MAPPING_INPUT_PATH) as file:
    gen_cat_file = file.read()

gen_cat_json = json.loads(gen_cat_file)
gen_cat_df = pd.DataFrame(list(gen_cat_json.items()), columns=['mapping', 'category'])

In [ ]:
len(gen_cat_df)

3432

In [ ]:
# Регулярное выражение для женских категорий
female_pattern = '|'.join(["женщин", "девуш", "девоч", "женск", "дам", "барышн", "государы"])
# Фильтрация для женских категорий
gen_cat_women_df = gen_cat_df[gen_cat_df['category'].str.lower().str.contains(female_pattern, na=False)]

# Регулярное выражение для мужских категорий
male_pattern = '|'.join(["мужч", "мальч", "мужск", "юнош", "господар"])
# Фильтрация для мужских категорий
gen_cat_men_df = gen_cat_df[gen_cat_df['category'].str.lower().str.contains(male_pattern, na=False)]

In [ ]:
gen_cat_men_df

,mapping,category
249,7812009,Для мальчиков
418,7815020,Обувь для мальчиков
597,90570,Электробритвы мужские
1091,8480712,Мужчинам
1261,10852916,Для мальчиков
2349,7811877,Мужская одежда
2364,7812193,Мужская обувь


In [ ]:
gen_cat_women_df

,mapping,category
240,7811873,Женская одежда
624,819013,Для лечения женских заболеваний
793,7815019,Обувь для девочек
859,10682603,Украшения для девочек
1010,90536,Женские контрацептивы
1525,7812192,Женская обувь
1633,91161,Эпиляторы и женские электробритвы
1956,10682538,Для девочек
2355,7812006,Для девочек


# Анализ данных

#### Просмотреть содержимое, частично

In [ ]:
first_five_pairs = list(train_json.items())[5:6]

for key, value in first_five_pairs:
    print(f"{key}: {value}")

user_6: {'target': 'female', 'features': {'orders': [{'site-id': 123, 'orders': [{'created-at': 1630352618, 'items': [{'id': 'item_75', 'count': 1, 'general-category-path': [7812065, 7812006, 7811879, 7877999], 'brand-id': 2458}, {'id': 'item_76', 'count': 1, 'general-category-path': [7811945, 7811896, 7811873, 7877999], 'brand-id': 2458}, {'id': 'item_77', 'count': 1, 'general-category-path': [7811945, 7811896, 7811873, 7877999], 'brand-id': 2458}]}, {'created-at': 1648999970, 'items': [{'id': 'item_78', 'count': 1, 'general-category-path': [7811945, 7811896, 7811873, 7877999], 'brand-id': 2458}, {'id': 'item_79', 'count': 1, 'general-category-path': [7811903, 7811873, 7877999], 'brand-id': 2458}]}, {'created-at': 1653315536, 'items': [{'id': 'item_80', 'count': 1, 'general-category-path': [7811945, 7811896, 7811873, 7877999], 'brand-id': 2458}, {'id': 'item_81', 'count': 1, 'general-category-path': [7812156, 7811877, 7877999], 'brand-id': 2458}]}]}, {'site-id': 50, 'orders': [{'creat

In [ ]:
train_draft_df = pd.json_normalize(train_json).T

In [ ]:
def collect_keys(d, parent_key='', sep='.'):
    keys = []
    for k, v in d.items():
        if not isinstance(k, str):
            continue

        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            keys.extend(collect_keys(v, new_key, sep=sep))
        elif isinstance(v, list):
            for item in v:
                if isinstance(item, dict):
                    keys.extend(collect_keys(item, new_key, sep=sep))
        else:
            keys.append(new_key)
    return keys

def remove_user_prefix(keys, pattern, sep='.'):
    processed_keys = []
    for key in keys:
        if isinstance(key, str):
            parts = key.split(sep)
            if re.match(pattern, parts[0]):
                processed_keys.append(sep.join(parts[1:]))
            else:
                processed_keys.append(key)
        elif isinstance(key, tuple):
            processed_key = tuple(remove_user_prefix(list(key), pattern))
            processed_keys.append(processed_key)
        else:
            processed_keys.append(key)
    return processed_keys

##### Количество уникальных ключей

In [ ]:
keys = collect_keys(train_json)
pattern = r'^user_\d+'
keys = remove_user_prefix(keys, pattern)
keys = list(set(keys))

In [ ]:
print(len(keys))

23


In [ ]:
keys.sort()
keys

['features.exchange-sessions.accepted-at',
 'features.exchange-sessions.accepted-site-id',
 'features.exchange-sessions.clicks.clicked-at',
 'features.exchange-sessions.clicks.site-id',
 'features.exchange-sessions.landed-at',
 'features.last-visits-in-categories.category',
 'features.last-visits-in-categories.last-visit-at',
 'features.orders.orders.created-at',
 'features.orders.orders.items.brand-id',
 'features.orders.orders.items.count',
 'features.orders.orders.items.id',
 'features.orders.site-id',
 'features.site-meta.frequency',
 'features.site-meta.monetary',
 'features.site-meta.recency',
 'features.site-meta.site-id',
 'features.visits.first-seen',
 'features.visits.last-seen',
 'features.visits.site-id',
 'features.visits.visits.pages-count',
 'features.visits.visits.session-duration',
 'features.visits.visits.visited-at',
 'target']

Черновик возможных параметров
orders:
 - общее количество заказов по всем сайтам вместе;
 - общее количество items во всех заказах по всем сайтам вместе;
 - общее количество items во всех заказах для каждого site;
 - общее количество items во всех заказах для каждого brand по всем сайтам вместе;
 - общее количество items во всех заказах для каждой category по всем сайтам вместе;
 - общее количество заказов из категорий для пола по всем сайтам вместе
 (7811873
819013
7815019
10682603
90536
7812192
91161
10682538
7812006) //женские товары
и
(7812009
7815020
90570
8480712
10852916
7811877
7812193) //мужские товары
site-meta:
	- общее количество сайтов в site-meta;
	- сумма frequency во всех site-meta;
	- средний frequency во всех site-meta;
	- сумма monetary во всех site-meta;
	- средней monetary во всех site-meta;
visits:
 - общее количество visits всех сайтов;
 - сумма pages-count по всем сайтам вместе;
 - среднее pages-count по всем сайтам вместе;
 - сумма session-duration по всем сайтам вместе;
 - среднее session-duration по всем сайтам вместе;
 - общее количество посещений для каждого brand по всем сайтам вместе;
 - среднее количество посещений для каждого brand по всем сайтам вместе;
 - общее число посещений всех category по всем сайтам вместе;
 - среднее число посещений всех category по всем сайтам вместе;
 - общее число посещений по каждой категории по всем сайтам вместе;
 - сумма session-duration по каждой категории по всем сайтам вместе;
 - среднее session-duration по каждой категории по всем сайтам вместе;
 - общее количество visits по каждому site;
 - сумма pages-count по каждому site;
 - среднее pages-count по каждому site;
 - сумма session-duration по каждому site;
 - среднее session-duration по каждому site;
 - общее количество заказов из категорий для пола по всем сайтам вместе
 (7811873
819013
7815019
10682603
90536
7812192
91161
10682538
7812006) //женские товары
и
(7812009
7815020
90570
8480712
10852916
7811877
7812193) //мужские товары

last-visits-in-categories:
 - общее число записей;
 - наличие записи (бинарная переменная для каждой категории).

In [ ]:
train_draft_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 698169 entries, user_1.target to user_127755.features.site-meta
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       698169 non-null  object
dtypes: object(1)
memory usage: 10.7+ MB


#### Популярные значения признаков в контексте заказов

In [ ]:
df_all_orders = pd.read_parquet(ALL_ORDERS_DF_PATH)

all_users_df = pd.DataFrame({'user': df_all_orders['user'].unique()})

df_all_orders['orders_count'] = df_all_orders['items'].apply(lambda x: len(x) if isinstance(x, list) else 0)

exploded_items = df_all_orders.explode('items')
items_df = pd.DataFrame(exploded_items['items'].dropna().tolist(), index=exploded_items['items'].dropna().index)
items_df = items_df.rename(columns={'id': 'item_id'})
items_df = items_df.merge(exploded_items[['user', 'site-id', 'orders_count']], left_index=True, right_index=True)
exploded_items = None

normalized_items = items_df.explode('general-category-path')

target_df = pd.DataFrame([(user, data['target']) for user, data in train_json.items()],
                        columns=['user', 'target'])

normalized_items = normalized_items.merge(target_df, on='user', how='left')

df_all_orders=None
all_users_df=None
exploded_items=None
items_df=None

In [ ]:
target_df.info()
normalized_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127755 entries, 0 to 127754
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   user    127755 non-null  object
 1   target  127755 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 58826811 entries, 0 to 58826810
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   brand-id               float64
 1   count                  float64
 2   general-category-path  object 
 3   item_id                object 
 4   user                   object 
 5   site-id                int64  
 6   orders_count           int64  
 7   target_x               object 
 8   target_y               object 
dtypes: float64(2), int64(2), object(5)
memory usage: 4.4+ GB


In [ ]:
def print_common_data_info(df):
    for column in ['site-id', 'brand-id', 'general-category-path']:
        unique_values_count = len(df[column].value_counts())

        zero_values_count = len(df[df[column] == 0])

        nan_values_count = df[column].isna().sum()
        nan_values_percent = (nan_values_count / len(df)) * 100

        total_count = len(df)
        zero_values_percent = (zero_values_count / total_count) * 100

        print(f"Stats for '{column}':")
        print(f"  Unique values count: {unique_values_count}")
        print(f"  Count of '0' values: {zero_values_count}")
        print(f"  Percentage of '0' values: {zero_values_percent:.2f}%")
        print(f"  Count of NaN values: {nan_values_count}")
        print(f"  Percentage of NaN values: {nan_values_percent:.2f}%")

def show_most_popular(df, target):
    def get_and_print_top_100(df, category, gender):
        gender_df = df[df[target] == gender]

        counts = gender_df[category].value_counts().nlargest(100)
        top_features = counts.index.tolist()
        total_count = gender_df[category].count()
        percentages = (counts / total_count) * 100

        print(f"Признак: {category}, Пол: {'мужской' if gender == 'male' else 'женский'}")
        print("Описательные статистики:")
        print(counts.describe())
        print("Топ-100:")
        print(counts)
        print("\nПроцент от общего числа:")
        print(percentages)
        print(f"\nОбщее количество для признака {category} для {'мужского' if gender == 'male' else 'женского'} пола: {total_count}")
        print("\n" + "-"*50 + "\n")

        return {'counts': counts, 'percentages': percentages, 'total_features': top_features, 'total_count': total_count}

    categories = ['brand-id', 'site-id', 'general-category-path']
    results = {category: {} for category in categories}

    for category in categories:
        for gender in ['male', 'female']:
            results[category][gender] = get_and_print_top_100(df, category, gender)

    return results

def merge_top_features(results):
    merged_top_features = {}
    for category in results:
        male_top_features = results[category]['male']['total_features']
        female_top_features = results[category]['female']['total_features']
        merged_top_features[category] = list(set(male_top_features + female_top_features))
    return merged_top_features

In [ ]:
print_common_data_info(normalized_items)
describe_stats = show_most_popular(normalized_items, 'target')
top_features = merge_top_features(describe_stats)

save_object_to_file(top_features, TOP_FEATURES_PATH)

Stats for 'site-id':
  Unique values count: 540
  Count of '0' values: 0
  Percentage of '0' values: 0.00%
  Count of NaN values: 0
  Percentage of NaN values: 0.00%
Stats for 'brand-id':
  Unique values count: 35489
  Count of '0' values: 0
  Percentage of '0' values: 0.00%
  Count of NaN values: 17289423
  Percentage of NaN values: 29.39%
Stats for 'general-category-path':
  Unique values count: 3321
  Count of '0' values: 0
  Percentage of '0' values: 0.00%
  Count of NaN values: 10552918
  Percentage of NaN values: 17.94%
Признак: brand-id, Пол: мужской
Описательные статистики:
count       100.000000
mean      37089.040000
std       40858.446749
min       16385.000000
25%       19909.750000
50%       26214.000000
75%       35319.250000
max      331576.000000
Name: brand-id, dtype: float64
Топ-100:
13333.0      331576
1183.0       227001
934.0        145054
15703.0       93359
3192.0        89231
15820.0       73981
2441.0        72279
13361.0       72129
13353.0       65810
2620.0 

In [ ]:
normalized_items.head(100)

,brand-id,count,general-category-path,item_id,user,site-id,orders_count,target
0,0.0,1.0,0,item_1,user_3,22,0,male
1,0.0,1.0,0,item_8,user_4,2,0,male
2,3243.0,1.0,90404,item_9,user_4,29,0,male
3,3243.0,1.0,90403,item_9,user_4,29,0,male
4,3243.0,1.0,90402,item_9,user_4,29,0,male
5,1210537.0,1.0,90698,item_10,user_4,29,0,male
6,1210537.0,1.0,7330336,item_10,user_4,29,0,male
7,1210537.0,1.0,90692,item_10,user_4,29,0,male
8,1210537.0,1.0,90666,item_10,user_4,29,0,male
9,1210537.0,1.0,90698,item_10,user_4,29,0,male


# Подготовка данных

## Подготовка метаданных сайтов (не заиспользована)

In [ ]:
def show_non_zero_data_for_user(df, user, comment):
    print(comment, '\n')

    user_row = df[df['user'] == user]
    user_row_non_na = user_row.dropna(axis=1)
    user_row_non_zero = user_row_non_na.loc[:, (user_row_non_na != 0).any(axis=0)]
    print(user_row_non_zero.head(5))

In [ ]:
def preprocess_site_meta(json_data):
    all_users_site_meta = []

    for user, user_data in json_data.items():
        if 'site-meta' in user_data.get('features', {}):
            df_user = pd.json_normalize(user_data, record_path=['features', 'site-meta'], errors='ignore')
            df_user['user'] = user

            df_user.fillna(0, inplace=True)

            all_users_site_meta.append(df_user)
        else:
            df_user = pd.DataFrame({'user': [user]})
            all_users_site_meta.append(df_user)

    df_all_users = pd.concat(all_users_site_meta)
    all_users_site_meta = None

    print(df_all_users.head(50))

    # трансформ
    site_ids_one_hot = pd.get_dummies(df_all_users['site-id'], prefix='site_id')
    df_all_users = pd.concat([df_all_users, site_ids_one_hot], axis=1)

    aggregated_data = df_all_users.groupby('user').agg({
        'site-id': 'nunique',
        'frequency': ['sum', 'mean'],
        'monetary': ['sum', 'mean']
    })

    one_hot_aggregated = df_all_users.groupby('user')[site_ids_one_hot.columns].sum()
    meta_pivot = pd.concat([aggregated_data, one_hot_aggregated], axis=1)
    site_ids_one_hot = None
    df_all_users = None
    aggregated_data = None
    one_hot_aggregated = None

    meta_pivot.columns = ['_'.join(col).strip() for col in meta_pivot.columns.values]
    meta_pivot.head()

    return meta_pivot

    site-id    user  recency  frequency  monetary
0       2.0  user_1      NaN        NaN       NaN
0       4.0  user_2      NaN        NaN       NaN
1       5.0  user_2      NaN        NaN       NaN
2       6.0  user_2      NaN        NaN       NaN
3       7.0  user_2      NaN        NaN       NaN
4       8.0  user_2      NaN        NaN       NaN
5       9.0  user_2      NaN        NaN       NaN
6      10.0  user_2      NaN        NaN       NaN
7      11.0  user_2      NaN        NaN       NaN
8      12.0  user_2      NaN        NaN       NaN
0      24.0  user_3      0.0        0.0       0.0
1      25.0  user_3      0.0        0.0       0.0
2      26.0  user_3      0.0        0.0       0.0
3      21.0  user_3      3.0        1.0       5.0
4      16.0  user_3      0.0        0.0       0.0
5      17.0  user_3      0.0        0.0       0.0
6      27.0  user_3      0.0        0.0       0.0
7      28.0  user_3      0.0        0.0       0.0
8      22.0  user_3      3.0        1.0       5.0


,site-id_nunique,frequency_sum,frequency_mean,monetary_sum,monetary_mean,s_i_t_e___i_d___1_._0,s_i_t_e___i_d___2_._0,s_i_t_e___i_d___3_._0,s_i_t_e___i_d___4_._0,s_i_t_e___i_d___5_._0,s_i_t_e___i_d___6_._0,s_i_t_e___i_d___7_._0,s_i_t_e___i_d___8_._0,s_i_t_e___i_d___9_._0,s_i_t_e___i_d___1_0_._0,s_i_t_e___i_d___1_1_._0,s_i_t_e___i_d___1_2_._0,s_i_t_e___i_d___1_3_._0,s_i_t_e___i_d___1_4_._0,s_i_t_e___i_d___1_5_._0,s_i_t_e___i_d___1_6_._0,s_i_t_e___i_d___1_7_._0,s_i_t_e___i_d___1_8_._0,s_i_t_e___i_d___1_9_._0,s_i_t_e___i_d___2_0_._0,s_i_t_e___i_d___2_1_._0,s_i_t_e___i_d___2_2_._0,s_i_t_e___i_d___2_3_._0,s_i_t_e___i_d___2_4_._0,s_i_t_e___i_d___2_5_._0,s_i_t_e___i_d___2_6_._0,s_i_t_e___i_d___2_7_._0,s_i_t_e___i_d___2_8_._0,s_i_t_e___i_d___2_9_._0,s_i_t_e___i_d___3_0_._0,s_i_t_e___i_d___3_1_._0,s_i_t_e___i_d___3_2_._0,s_i_t_e___i_d___3_3_._0,s_i_t_e___i_d___3_4_._0,s_i_t_e___i_d___3_5_._0,s_i_t_e___i_d___3_6_._0,s_i_t_e___i_d___3_7_._0,s_i_t_e___i_d___3_8_._0,s_i_t_e___i_d___3_9_._0,s_i_t_e___i_d___4_0_._0,s_i_t_e___i_d___4_1_._0,s_i_t_e___i_d___4_2_._0,s_i_t_e___i_d___4_3_._0,s_i_t_e___i_d___4_4_._0,s_i_t_e___i_d___4_5_._0,s_i_t_e___i_d___4_6_._0,s_i_t_e___i_d___4_7_._0,s_i_t_e___i_d___4_8_._0,s_i_t_e___i_d___4_9_._0,s_i_t_e___i_d___5_0_._0,s_i_t_e___i_d___5_1_._0,s_i_t_e___i_d___5_3_._0,s_i_t_e___i_d___5_4_._0,s_i_t_e___i_d___5_5_._0,s_i_t_e___i_d___5_6_._0,s_i_t_e___i_d___5_7_._0,s_i_t_e___i_d___5_8_._0,s_i_t_e___i_d___5_9_._0,s_i_t_e___i_d___6_0_._0,s_i_t_e___i_d___6_1_._0,s_i_t_e___i_d___6_2_._0,s_i_t_e___i_d___6_3_._0,s_i_t_e___i_d___6_4_._0,s_i_t_e___i_d___6_5_._0,s_i_t_e___i_d___6_6_._0,s_i_t_e___i_d___6_7_._0,s_i_t_e___i_d___6_8_._0,s_i_t_e___i_d___6_9_._0,s_i_t_e___i_d___7_0_._0,s_i_t_e___i_d___7_1_._0,s_i_t_e___i_d___7_3_._0,s_i_t_e___i_d___7_4_._0,s_i_t_e___i_d___7_6_._0,s_i_t_e___i_d___7_7_._0,s_i_t_e___i_d___7_8_._0,s_i_t_e___i_d___7_9_._0,s_i_t_e___i_d___8_0_._0,s_i_t_e___i_d___8_1_._0,s_i_t_e___i_d___8_2_._0,s_i_t_e___i_d___8_3_._0,s_i_t_e___i_d___8_4_._0,s_i_t_e___i_d___8_5_._0,s_i_t_e___i_d___8_6_._0,s_i_t_e___i_d___8_7_._0,s_i_t_e___i_d___8_8_._0,s_i_t_e___i_d___8_9_._0,s_i_t_e___i_d___9_0_._0,s_i_t_e___i_d___9_1_._0,s_i_t_e___i_d___9_2_._0,s_i_t_e___i_d___9_3_._0,s_i_t_e___i_d___9_4_._0,s_i_t_e___i_d___9_5_._0,s_i_t_e___i_d___9_6_._0,s_i_t_e___i_d___9_7_._0,s_i_t_e___i_d___9_8_._0,s_i_t_e___i_d___9_9_._0,s_i_t_e___i_d___1_0_0_._0,s_i_t_e___i_d___1_0_1_._0,s_i_t_e___i_d___1_0_2_._0,s_i_t_e___i_d___1_0_3_._0,s_i_t_e___i_d___1_0_4_._0,s_i_t_e___i_d___1_0_5_._0,s_i_t_e___i_d___1_0_6_._0,s_i_t_e___i_d___1_0_7_._0,s_i_t_e___i_d___1_0_8_._0,s_i_t_e___i_d___1_0_9_._0,s_i_t_e___i_d___1_1_0_._0,s_i_t_e___i_d___1_1_1_._0,s_i_t_e___i_d___1_1_2_._0,s_i_t_e___i_d___1_1_3_._0,s_i_t_e___i_d___1_1_4_._0,s_i_t_e___i_d___1_1_5_._0,s_i_t_e___i_d___1_1_6_._0,s_i_t_e___i_d___1_1_7_._0,s_i_t_e___i_d___1_1_8_._0,s_i_t_e___i_d___1_1_9_._0,s_i_t_e___i_d___1_2_0_._0,s_i_t_e___i_d___1_2_1_._0,s_i_t_e___i_d___1_2_2_._0,s_i_t_e___i_d___1_2_3_._0,s_i_t_e___i_d___1_2_4_._0,s_i_t_e___i_d___1_2_7_._0,s_i_t_e___i_d___1_2_8_._0,s_i_t_e___i_d___1_2_9_._0,s_i_t_e___i_d___1_3_0_._0,s_i_t_e___i_d___1_3_2_._0,s_i_t_e___i_d___1_3_3_._0,s_i_t_e___i_d___1_3_4_._0,s_i_t_e___i_d___1_3_5_._0,s_i_t_e___i_d___1_3_6_._0,s_i_t_e___i_d___1_3_7_._0,s_i_t_e___i_d___1_3_8_._0,s_i_t_e___i_d___1_3_9_._0,s_i_t_e___i_d___1_4_0_._0,s_i_t_e___i_d___1_4_1_._0,s_i_t_e___i_d___1_4_2_._0,s_i_t_e___i_d___1_4_3_._0,s_i_t_e___i_d___1_4_4_._0,s_i_t_e___i_d___1_4_5_._0,s_i_t_e___i_d___1_4_6_._0,s_i_t_e___i_d___1_4_7_._0,s_i_t_e___i_d___1_4_8_._0,s_i_t_e___i_d___1_4_9_._0,s_i_t_e___i_d___1_5_0_._0,s_i_t_e___i_d___1_5_1_._0,s_i_t_e___i_d___1_5_2_._0,s_i_t_e___i_d___1_5_3_._0,s_i_t_e___i_d___1_5_4_._0,s_i_t_e___i_d___1_5_5_._0,s_i_t_e___i_d___1_5_6_._0,s_i_t_e___i_d___1_5_7_._0,s_i_t_e___i_d___1_5_8_._0,s_i_t_e___i_d___1_5_9_._0,s_i_t_e___i_d___1_6_0_._0,s_i_t_e___i_d___1_6_1_._0,s_i_t_e___i_d___1_6_2_._0,s_i_t_e___i_d___1_6_3_._0,s_i_t_e___i_d___1_6_4_._0,s_i_t_e___i_d___1_6_5_._0,s_i

In [ ]:
meta_pivot = preprocess_site_meta(train_json)

In [ ]:
meta_pivot.to_parquet(META_DF_PATH, compression='gzip')

## Подготовка обучающего набора (по заказам)

In [ ]:
def preprocess_orders(json_data, orders_per_site_matr_path, orders_per_site_columns_path, orders_per_site_index_path,
                      items_per_site_matr_path, items_per_site_columns_path, items_per_site_index_path,
                      item_per_brand_matr_path, item_per_brand_columns_path, item_per_brand_index_path,
                      items_per_category_matr_path, items_per_category_columns_path, items_per_category_index_path,
                      top_features,
                      load_normalized_from_fs=True, get_orders_per_site=True, get_items_count_per_site=True,
                      get_items_count_per_brand=True, get_items_count_per_category=True, from_threshold=0, to_threshold=1, load_normalized_from_fs_path=ALL_ORDERS_DF_PATH):

    def rename_columns_with_id(pivot_table, prefix):
        return pivot_table.rename(columns=lambda x: f"{prefix}_{x}" if not pd.isna(x) else x)

    def downcast_pivot(pivot_table):
        for col in pivot_table.columns:
            pivot_table[col] = pd.to_numeric(pivot_table[col], downcast='integer')
        return pivot_table

    def save_sparse_dataframe(sparse_matrix, columns, index, matrix_path, columns_path, index_path):
        save_npz(matrix_path, sparse_matrix)
        pd.Series(columns).to_csv(columns_path, index=False, header=False)
        pd.Series(index).to_csv(index_path, index=False, header=False)

    def create_and_downcast_pivot(grouped_df, index_col, columns_col, values_col, prefix, all_users_df, df_name, matrix_path, columns_path, index_path, top_features):
        grouped_df = grouped_df[grouped_df[columns_col].isin(top_features)]
        grouped_df.set_index(index_col, inplace=True)
        all_combinations = all_users_df.assign(key=1).merge(pd.DataFrame({columns_col: grouped_df[columns_col].unique(), 'key': 1}), on='key').drop('key', 1)
        all_combinations = all_combinations.merge(grouped_df, on=[index_col, columns_col], how='left').fillna(0)
        grouped_df = None
        print(f'{df_name} shape: ', all_combinations.shape)
        pivot_table = all_combinations.pivot_table(index=index_col, columns=columns_col, values=values_col, fill_value=0, aggfunc='sum')
        all_combinations = None
        print(f'{df_name} - созданы признаки, в процессе дообработка и сохранение..')
        pivot_table = downcast_pivot(pivot_table)
        pivot_table = pivot_table.rename(columns=lambda x: f"{prefix}_{x}" if not pd.isna(x) else x)
        columns_values = pivot_table.columns.values

        for col in pivot_table.columns:
            if pd.api.types.is_numeric_dtype(pivot_table[col]):
                pivot_table[col] = pd.arrays.SparseArray(pivot_table[col], fill_value=0)

        pivot_index = pivot_table.index
        pivot_table = pivot_table.sparse.to_coo().tocsr()

        save_sparse_dataframe(pivot_table, columns_values, pivot_index, matrix_path, columns_path, index_path)
        pivot_table = None

    df_all_orders = None
    if load_normalized_from_fs:
        df_all_orders = pd.read_parquet(load_normalized_from_fs_path)
    else:
        all_orders_data = []
        for user, user_data in json_data.items():
            if 'features' in user_data and 'orders' in user_data['features']:
                for order in user_data['features']['orders']:
                    df_order = pd.json_normalize(
                        order,
                        record_path='orders',
                        meta=['site-id'],
                        errors='ignore'
                    )

                    df_order['user'] = user
                    all_orders_data.append(df_order)
                    df_order = None
        df_all_orders = pd.concat(all_orders_data, ignore_index=True)
        all_orders_data = None

    json_data = None
    train_json = None

    if not load_normalized_from_fs:
        df_all_orders.to_parquet(load_normalized_from_fs_path, compression='gzip')

    from_index = int(from_threshold * len(df_all_orders))
    to_index = int(to_threshold * len(df_all_orders))
    print(f"Processing from index {from_index} to {to_index} of total len {len(df_all_orders)}")
    df_all_orders = df_all_orders.iloc[from_index:to_index]

    all_users_df = pd.DataFrame({'user': df_all_orders['user'].unique()})

    df_all_orders['orders_count'] = df_all_orders['items'].apply(lambda x: len(x) if isinstance(x, list) else 0)

    exploded_items = df_all_orders.explode('items')
    items_df = pd.DataFrame(exploded_items['items'].dropna().tolist(), index=exploded_items['items'].dropna().index)
    items_df = items_df.rename(columns={'id': 'item_id'})
    items_df = items_df.merge(exploded_items[['user', 'site-id', 'orders_count']], left_index=True, right_index=True)
    exploded_items = None

    normalized_items = items_df.explode('general-category-path')
    columns_to_check = ['site-id', 'brand-id', 'general-category-path']
    normalized_items = normalized_items.dropna(subset=columns_to_check)

    items_df = None

    if get_orders_per_site:
        top_sites = top_features['site-id']
        orders_per_site = df_all_orders.groupby(['user', 'site-id']).size().reset_index(name='orders_count_per_site')
        create_and_downcast_pivot(orders_per_site, 'user', 'site-id', 'orders_count_per_site', 'orders_site', all_users_df, 'pivot_orders_per_site',
                                  orders_per_site_matr_path, orders_per_site_columns_path, orders_per_site_index_path, top_sites)
        orders_per_site = None
        df_all_orders = None
        print('orders_per_site - success')

    if get_items_count_per_site:
        top_sites = top_features['site-id']
        items_count_per_site = normalized_items.groupby(['user', 'site-id'])['item_id'].nunique().reset_index(name='unique_items_count_per_site')
        create_and_downcast_pivot(items_count_per_site, 'user', 'site-id', 'unique_items_count_per_site', 'items_site', all_users_df, 'pivot_items_per_site',
                                  items_per_site_matr_path, items_per_site_columns_path, items_per_site_index_path, top_sites)
        items_count_per_site = None
        print('pivot_items_per_site - success')

    if get_items_count_per_brand:
        top_brands = top_features['brand-id']
        items_count_per_brand = normalized_items.groupby(['user', 'brand-id'])['item_id'].nunique().reset_index(name='unique_items_count_per_brand')
        create_and_downcast_pivot(items_count_per_brand, 'user', 'brand-id', 'unique_items_count_per_brand', 'items_brand', all_users_df, 'pivot_items_per_brand',
                                  item_per_brand_matr_path, item_per_brand_columns_path, item_per_brand_index_path, top_brands)
        items_count_per_brand = None
        print('pivot_items_per_brand - success')

    if get_items_count_per_category:
        top_category = top_features['general-category-path']
        items_count_per_category = normalized_items.groupby(['user', 'general-category-path'])['item_id'].nunique().reset_index(name='unique_items_count_per_category')
        normalized_items = None
        create_and_downcast_pivot(items_count_per_category, 'user', 'general-category-path', 'unique_items_count_per_category', 'items_category', all_users_df, 'pivot_items_per_category',
                                  items_per_category_matr_path, items_per_category_columns_path, items_per_category_index_path, top_category)
        items_count_per_category = None
        print('pivot_items_per_category - success')

    return all_users_df

#### Предобработать и записать в файловую систему

In [ ]:
top_features = load_object_from_file(TOP_FEATURES_PATH)

In [ ]:
top_features

{'brand-id': [16384.0,
  1.0,
  2.0,
  6.0,
  9735.0,
  7.0,
  15886.0,
  15888.0,
  12304.0,
  13332.0,
  13333.0,
  13334.0,
  13335.0,
  15896.0,
  13337.0,
  13339.0,
  1055.0,
  13343.0,
  13345.0,
  13348.0,
  13353.0,
  4650.0,
  102956.0,
  559.0,
  13361.0,
  13363.0,
  124471.0,
  2620.0,
  13374.0,
  12355.0,
  9801.0,
  590.0,
  594.0,
  1210459.0,
  605.0,
  607.0,
  15456.0,
  15457.0,
  610.0,
  11874.0,
  1635.0,
  10342.0,
  26216.0,
  26226.0,
  3192.0,
  11899.0,
  4733.0,
  11908.0,
  19594.0,
  29335.0,
  30364.0,
  1183.0,
  2208.0,
  12462.0,
  10417.0,
  696.0,
  4793.0,
  702.0,
  18115.0,
  4807.0,
  4811.0,
  2254.0,
  29394.0,
  11998.0,
  4836.0,
  12518.0,
  742.0,
  3304.0,
  4841.0,
  4847.0,
  12017.0,
  5878.0,
  18180.0,
  777.0,
  3850.0,
  3854.0,
  12050.0,
  16660.0,
  798.0,
  25393.0,
  12086.0,
  824.0,
  828.0,
  16700.0,
  829.0,
  837.0,
  15699.0,
  1878.0,
  15703.0,
  1558358.0,
  15713.0,
  15719.0,
  14711.0,
  2441.0,
  16778.0,
  2292

In [ ]:
all_users_df_1 = preprocess_orders(train_json, ORDERS_PER_SITE_MATR_PATH_1, ORDERS_PER_SITE_COLUMNS_PATH_1, ORDERS_PER_SITE_INDEX_PATH_1,
                                 ITEMS_PER_SITE_MATR_PATH_1, ITEMS_PER_SITE_COLUMNS_PATH_1, ITEMS_PER_SITE_INDEX_PATH_1,
                                 ITEM_PER_BRAND_MATR_PATH_1, ITEM_PER_BRAND_COLUMNS_PATH_1, ITEM_PER_BRAND_INDEX_PATH_1,
                                 ITEMS_PER_CATEGORY_MATR_PATH_1, ITEMS_PER_CATEGORY_COLUMNS_PATH_1, ITEMS_PER_CATEGORY_INDEX_PATH_1,
                                 top_features,
                                 load_normalized_from_fs=True, get_orders_per_site=True, get_items_count_per_site=True,
                                 get_items_count_per_brand=True, get_items_count_per_category=True, from_threshold=0, to_threshold=1)

Processing from index 0 to 1622283 of total len 1622283


<ipython-input-8-fc4e2b186eb4>:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  all_combinations = all_users_df.assign(key=1).merge(pd.DataFrame({columns_col: grouped_df[columns_col].unique(), 'key': 1}), on='key').drop('key', 1)


pivot_orders_per_site shape:  (13976589, 3)
orders_per_site - success


<ipython-input-8-fc4e2b186eb4>:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  all_combinations = all_users_df.assign(key=1).merge(pd.DataFrame({columns_col: grouped_df[columns_col].unique(), 'key': 1}), on='key').drop('key', 1)


pivot_items_per_site shape:  (10742337, 3)
pivot_items_per_site - success


<ipython-input-8-fc4e2b186eb4>:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  all_combinations = all_users_df.assign(key=1).merge(pd.DataFrame({columns_col: grouped_df[columns_col].unique(), 'key': 1}), on='key').drop('key', 1)


pivot_items_per_brand shape:  (14669643, 3)
pivot_items_per_brand - success


<ipython-input-8-fc4e2b186eb4>:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  all_combinations = all_users_df.assign(key=1).merge(pd.DataFrame({columns_col: grouped_df[columns_col].unique(), 'key': 1}), on='key').drop('key', 1)


pivot_items_per_category shape:  (13745571, 3)
pivot_items_per_category - success


#### Предобработать и записать в файловую систему части основного набора (опциональный шаг)

In [ ]:
# Процессинг второй половины заказов тестового набора
all_users_df_2 = preprocess_orders(train_json, ORDERS_PER_SITE_MATR_PATH_2, ORDERS_PER_SITE_COLUMNS_PATH_2, ORDERS_PER_SITE_INDEX_PATH_2,
                                 ITEMS_PER_SITE_MATR_PATH_2, ITEMS_PER_SITE_COLUMNS_PATH_2, ITEMS_PER_SITE_INDEX_PATH_2,
                                 ITEM_PER_BRAND_MATR_PATH_2, ITEM_PER_BRAND_COLUMNS_PATH_2, ITEM_PER_BRAND_INDEX_PATH_2,
                                 ITEMS_PER_CATEGORY_MATR_PATH_2, ITEMS_PER_CATEGORY_COLUMNS_PATH_2, ITEMS_PER_CATEGORY_INDEX_PATH_2,
                                 load_normalized_from_fs=True, get_orders_per_site=True, get_items_count_per_site=True,
                                 get_items_count_per_brand=True, get_items_count_per_category=True, from_threshold=0.5, to_threshold=1)

In [ ]:

all_users_df_3 = preprocess_orders(train_json, ORDERS_PER_SITE_MATR_PATH_3, ORDERS_PER_SITE_COLUMNS_PATH_3, ORDERS_PER_SITE_INDEX_PATH_3,
                                 ITEMS_PER_SITE_MATR_PATH_3, ITEMS_PER_SITE_COLUMNS_PATH_3, ITEMS_PER_SITE_INDEX_PATH_3,
                                 ITEM_PER_BRAND_MATR_PATH_3, ITEM_PER_BRAND_COLUMNS_PATH_3, ITEM_PER_BRAND_INDEX_PATH_3,
                                 ITEMS_PER_CATEGORY_MATR_PATH_3, ITEMS_PER_CATEGORY_COLUMNS_PATH_3, ITEMS_PER_CATEGORY_INDEX_PATH_3,
                                 load_normalized_from_fs=True, get_orders_per_site=True, get_items_count_per_site=True,
                                 get_items_count_per_brand=True, get_items_count_per_category=True, from_index=THRESHOLD_3_4, to_index=None)

#### Загрузить и объединить квартет наборов

In [ ]:
def merge_df_quartet(orders_per_site_matr_path, orders_per_site_columns_path, orders_per_site_index_path,
                      items_per_site_matr_path, items_per_site_columns_path, items_per_site_index_path,
                      item_per_brand_matr_path, item_per_brand_columns_path, item_per_brand_index_path,
                      items_per_category_matr_path, items_per_category_columns_path, items_per_category_index_path):
    def load_sparse_dataframe(matrix_path, columns_path, index_path):
        sparse_matrix = load_npz(matrix_path)
        columns = pd.read_csv(columns_path, header=None).squeeze()
        index = pd.read_csv(index_path, header=None).squeeze()
        return sparse_matrix, columns, index

    pivot_orders_per_site, pivot_orders_per_site_columns, pivot_orders_per_site_index = load_sparse_dataframe(orders_per_site_matr_path, orders_per_site_columns_path, orders_per_site_index_path)
    pivot_items_per_site, pivot_items_per_site_columns, pivot_items_per_site_index = load_sparse_dataframe(items_per_site_matr_path, items_per_site_columns_path, items_per_site_index_path)
    pivot_items_per_brand, pivot_items_per_brand_columns, pivot_items_per_brand_index = load_sparse_dataframe(item_per_brand_matr_path, item_per_brand_columns_path, item_per_brand_index_path)
    pivot_items_per_category, pivot_items_per_category_columns, pivot_items_per_category_index = load_sparse_dataframe(items_per_category_matr_path, items_per_category_columns_path, items_per_category_index_path)

    order_pivot = hstack([pivot_orders_per_site, pivot_items_per_site, pivot_items_per_brand, pivot_items_per_category])

    pivot_orders_per_site = None
    pivot_items_per_site = None
    pivot_items_per_brand = None
    pivot_items_per_category = None

    column_names = np.concatenate([
        pivot_orders_per_site_columns,
        pivot_items_per_site_columns,
        pivot_items_per_brand_columns,
        pivot_items_per_category_columns
    ])

    order_pivot = pd.DataFrame.sparse.from_spmatrix(order_pivot)
    order_pivot.index = pivot_orders_per_site_index
    order_pivot.columns = column_names

    print("unique users:", len(order_pivot.index.unique()))
    print("order_pivot_df shape:", order_pivot.shape)
    print(order_pivot.info())

    return order_pivot

In [ ]:
order_pivot_train = merge_df_quartet(ORDERS_PER_SITE_MATR_PATH_1, ORDERS_PER_SITE_COLUMNS_PATH_1, ORDERS_PER_SITE_INDEX_PATH_1,
                 ITEMS_PER_SITE_MATR_PATH_1, ITEMS_PER_SITE_COLUMNS_PATH_1, ITEMS_PER_SITE_INDEX_PATH_1,
                 ITEM_PER_BRAND_MATR_PATH_1, ITEM_PER_BRAND_COLUMNS_PATH_1, ITEM_PER_BRAND_INDEX_PATH_1,
                 ITEMS_PER_CATEGORY_MATR_PATH_1, ITEMS_PER_CATEGORY_COLUMNS_PATH_1, ITEMS_PER_CATEGORY_INDEX_PATH_1)

unique users: 115509
order_pivot_df shape: (115509, 460)
<class 'pandas.core.frame.DataFrame'>
Index: 115509 entries, user_1 to user_99999
Columns: 460 entries, orders_site_2 to items_category_16044336
dtypes: Sparse[int16, 0](460)
memory usage: 13.3+ MB
None


#### Загрузить и объединить прочие квартеты наборов (опциональный шаг)

In [ ]:
# не использовался
order_pivot_2 = merge_df_quartet(ORDERS_PER_SITE_MATR_PATH_2, ORDERS_PER_SITE_COLUMNS_PATH_2, ORDERS_PER_SITE_INDEX_PATH_2,
                                 ITEMS_PER_SITE_MATR_PATH_2, ITEMS_PER_SITE_COLUMNS_PATH_2, ITEMS_PER_SITE_INDEX_PATH_2,
                                 ITEM_PER_BRAND_MATR_PATH_2, ITEM_PER_BRAND_COLUMNS_PATH_2, ITEM_PER_BRAND_INDEX_PATH_2,
                                 ITEMS_PER_CATEGORY_MATR_PATH_2, ITEMS_PER_CATEGORY_COLUMNS_PATH_2, ITEMS_PER_CATEGORY_INDEX_PATH_2)

#### Склеить (при наличии множества квартетов наборов) - опциональный шаг


In [ ]:
# не использовался
def align_and_concatenate_dfs(upper_df, lower_df):
    all_columns = sorted(set(upper_df.columns).union(set(lower_df.columns)))

    upper_df_aligned = upper_df.reindex(columns=all_columns, fill_value=0)
    lower_df_aligned = lower_df.reindex(columns=all_columns, fill_value=0)

    concatenated_df = pd.concat([lower_df_aligned, upper_df_aligned])
    return concatenated_df

In [ ]:
# не использовался
order_pivot = align_and_concatenate_dfs(order_pivot_1, order_pivot_2)

#### Проверить набор

In [ ]:
def smoke_test(df, user):
    user_index = user
    user_row = df.loc[user]
    non_zero_columns = user_row[user_row != 0]
    non_zero_non_na_columns = non_zero_columns.dropna()
    print(non_zero_non_na_columns)

In [ ]:
smoke_test(order_pivot_train, 'user_6')

orders_site_21              2
orders_site_22              2
orders_site_50              8
orders_site_146             1
orders_site_151             3
items_site_22               1
items_site_50              15
items_site_146             10
items_site_151              3
items_brand_1423.0          1
items_brand_3854.0          2
items_category_90509       13
items_category_90666        3
items_category_90719        1
items_category_91157       10
items_category_91159        3
items_category_91172        3
items_category_91182        2
items_category_91183        2
items_category_91186        1
items_category_91307        1
items_category_91325        1
items_category_91391        1
items_category_91512        1
items_category_91597        1
items_category_198118       1
items_category_819000      10
items_category_1618974      1
items_category_4748054      3
items_category_6179129      1
items_category_7811873      5
items_category_7811877      1
items_category_7811879      1
items_cate

In [ ]:
save_object_to_file(order_pivot_train, TRAIN_PROCESSED)

Пусть сохранения /content/drive/MyDrive/docs/keepForever/mipt/flactory_dataton/df/train_preprocessed_df.pkl


## Подготовка val набора

In [ ]:
with open(VAL_INPUT_PATH) as file:
    val_file = file.read()

val_json = json.loads(val_file)

In [ ]:
# процессинг заказов валидационного набора
all_users_val_df = preprocess_orders(val_json, ORDERS_PER_SITE_MATR_PATH_VAL, ORDERS_PER_SITE_COLUMNS_PATH_VAL, ORDERS_PER_SITE_INDEX_PATH_VAL,
                                    ITEMS_PER_SITE_MATR_PATH_VAL, ITEMS_PER_SITE_COLUMNS_PATH_VAL, ITEMS_PER_SITE_INDEX_PATH_VAL,
                                    ITEM_PER_BRAND_MATR_PATH_VAL, ITEM_PER_BRAND_COLUMNS_PATH_VAL, ITEM_PER_BRAND_INDEX_PATH_VAL,
                                    ITEMS_PER_CATEGORY_MATR_PATH_VAL, ITEMS_PER_CATEGORY_COLUMNS_PATH_VAL, ITEMS_PER_CATEGORY_INDEX_PATH_VAL,
                                    top_features,
                                    load_normalized_from_fs=True, get_orders_per_site=True, get_items_count_per_site=True,
                                    get_items_count_per_brand=True, get_items_count_per_category=True, from_threshold=0, to_threshold=1, load_normalized_from_fs_path=ALL_ORDERS_VAL_DF_PATH)

In [ ]:
order_pivot_val = merge_df_quartet(ORDERS_PER_SITE_MATR_PATH_VAL, ORDERS_PER_SITE_COLUMNS_PATH_VAL, ORDERS_PER_SITE_INDEX_PATH_VAL,
                                  ITEMS_PER_SITE_MATR_PATH_VAL, ITEMS_PER_SITE_COLUMNS_PATH_VAL, ITEMS_PER_SITE_INDEX_PATH_VAL,
                                  ITEM_PER_BRAND_MATR_PATH_VAL, ITEM_PER_BRAND_COLUMNS_PATH_VAL, ITEM_PER_BRAND_INDEX_PATH_VAL,
                                  ITEMS_PER_CATEGORY_MATR_PATH_VAL, ITEMS_PER_CATEGORY_COLUMNS_PATH_VAL, ITEMS_PER_CATEGORY_INDEX_PATH_VAL)

unique users: 24871
order_pivot_df shape: (24871, 459)
<class 'pandas.core.frame.DataFrame'>
Index: 24871 entries, user_127756 to user_155202
Columns: 459 entries, orders_site_2 to items_category_16044336
dtypes: Sparse[int16, 0](459)
memory usage: 2.6+ MB
None


In [ ]:
result_with_labels_val = order_pivot_val.iloc[:5, :10]

print(result_with_labels_val)

             orders_site_2  orders_site_5  orders_site_6  orders_site_8  orders_site_11  \
0                                                                                         
user_127756              0              0              0              0               0   
user_127757              0              0              0              0               0   
user_127758              0              0              0              0               0   
user_127759              0              0              0              0               0   
user_127761              0              0              0              0               0   

             orders_site_12  orders_site_16  orders_site_18  orders_site_20  orders_site_21  
0                                                                                            
user_127756               0               0               0               0               0  
user_127757               0              10               0               0     

In [ ]:
smoke_test(order_pivot_val, 'user_127757')

orders_site_16            10
orders_site_21             2
orders_site_40             1
items_site_16              9
items_brand_1.0            2
items_brand_1055.0         3
items_category_90666       2
items_category_90692       2
items_category_91461       3
items_category_198118      5
items_category_198119      4
items_category_7330336     1
Name: user_127757, dtype: Sparse[int16, 0]


In [ ]:
save_object_to_file(order_pivot_val, VAL_PROCESSED)

Пусть сохранения /content/drive/MyDrive/docs/keepForever/mipt/flactory_dataton/df/val_preprocessed_df.pkl


## Подготовка тестового набора

In [ ]:
with open(TEST_INPUT_PATH) as file:
    test_file = file.read()

test_json = json.loads(test_file)

In [ ]:
# процессинг заказов тестового набора
all_users_test_df = preprocess_orders(test_json, ORDERS_PER_SITE_MATR_PATH_TEST, ORDERS_PER_SITE_COLUMNS_PATH_TEST, ORDERS_PER_SITE_INDEX_PATH_TEST,
                                    ITEMS_PER_SITE_MATR_PATH_TEST, ITEMS_PER_SITE_COLUMNS_PATH_TEST, ITEMS_PER_SITE_INDEX_PATH_TEST,
                                    ITEM_PER_BRAND_MATR_PATH_TEST, ITEM_PER_BRAND_COLUMNS_PATH_TEST, ITEM_PER_BRAND_INDEX_PATH_TEST,
                                    ITEMS_PER_CATEGORY_MATR_PATH_TEST, ITEMS_PER_CATEGORY_COLUMNS_PATH_TEST, ITEMS_PER_CATEGORY_INDEX_PATH_TEST,
                                    top_features,
                                    load_normalized_from_fs=True, get_orders_per_site=True, get_items_count_per_site=True,
                                    get_items_count_per_brand=True, get_items_count_per_category=True, from_threshold=0, to_threshold=1, load_normalized_from_fs_path=ALL_ORDERS_TEST_DF_PATH)

Processing from index 0 to 252298 of total len 252298


<ipython-input-11-fa3393760b78>:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  all_combinations = all_users_df.assign(key=1).merge(pd.DataFrame({columns_col: grouped_df[columns_col].unique(), 'key': 1}), on='key').drop('key', 1)


pivot_orders_per_site shape:  (2047562, 3)
pivot_orders_per_site - созданы признаки, в процессе дообработка и сохранение..
orders_per_site - success


<ipython-input-11-fa3393760b78>:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  all_combinations = all_users_df.assign(key=1).merge(pd.DataFrame({columns_col: grouped_df[columns_col].unique(), 'key': 1}), on='key').drop('key', 1)


pivot_items_per_site shape:  (1556824, 3)
pivot_items_per_site - созданы признаки, в процессе дообработка и сохранение..
pivot_items_per_site - success


<ipython-input-11-fa3393760b78>:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  all_combinations = all_users_df.assign(key=1).merge(pd.DataFrame({columns_col: grouped_df[columns_col].unique(), 'key': 1}), on='key').drop('key', 1)


pivot_items_per_brand shape:  (2149094, 3)
pivot_items_per_brand - созданы признаки, в процессе дообработка и сохранение..
pivot_items_per_brand - success


<ipython-input-11-fa3393760b78>:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  all_combinations = all_users_df.assign(key=1).merge(pd.DataFrame({columns_col: grouped_df[columns_col].unique(), 'key': 1}), on='key').drop('key', 1)


pivot_items_per_category shape:  (2013718, 3)
pivot_items_per_category - созданы признаки, в процессе дообработка и сохранение..
pivot_items_per_category - success


In [ ]:
order_pivot_test = merge_df_quartet(ORDERS_PER_SITE_MATR_PATH_TEST, ORDERS_PER_SITE_COLUMNS_PATH_TEST, ORDERS_PER_SITE_INDEX_PATH_TEST,
                                  ITEMS_PER_SITE_MATR_PATH_TEST, ITEMS_PER_SITE_COLUMNS_PATH_TEST, ITEMS_PER_SITE_INDEX_PATH_TEST,
                                  ITEM_PER_BRAND_MATR_PATH_TEST, ITEM_PER_BRAND_COLUMNS_PATH_TEST, ITEM_PER_BRAND_INDEX_PATH_TEST,
                                  ITEMS_PER_CATEGORY_MATR_PATH_TEST, ITEMS_PER_CATEGORY_COLUMNS_PATH_TEST, ITEMS_PER_CATEGORY_INDEX_PATH_TEST)

unique users: 16922
order_pivot_df shape: (16922, 459)
<class 'pandas.core.frame.DataFrame'>
Index: 16922 entries, user_155204 to user_173986
Columns: 459 entries, orders_site_2 to items_category_16044336
dtypes: Sparse[int16, 0](459)
memory usage: 1.9+ MB
None


In [ ]:
result_with_labels_test = order_pivot_test.iloc[:5, :10]

print(result_with_labels_test)

             orders_site_2  orders_site_5  orders_site_6  orders_site_8  orders_site_11  \
0                                                                                         
user_155204              0              0              0              0               0   
user_155206              0              0              0              1               0   
user_155207              0              0              0              0               0   
user_155208              0              0              0             10               0   
user_155209              0              0              0              0               0   

             orders_site_12  orders_site_16  orders_site_18  orders_site_20  orders_site_21  
0                                                                                            
user_155204               0               0               0               0               0  
user_155206               0               0               0               8     

In [ ]:
smoke_test(order_pivot_test, 'user_155208')

orders_site_8              10
orders_site_16              3
orders_site_20             10
orders_site_21              2
orders_site_29              1
orders_site_46             10
orders_site_50              2
orders_site_112             4
orders_site_179             2
orders_site_184             1
items_site_8               21
items_site_20              56
items_site_21               4
items_site_29               1
items_site_50               1
items_site_112              3
items_site_184              1
items_brand_559.0           1
items_brand_934.0           5
items_brand_1183.0         10
items_brand_12355.0         1
items_brand_13334.0         1
items_brand_15888.0         1
items_brand_16384.0         1
items_brand_18115.0         1
items_brand_19594.0         1
items_category_90402        1
items_category_90509        7
items_category_90666        6
items_category_90692        1
items_category_90748        2
items_category_90764       13
items_category_90783        1
items_cate

In [ ]:
save_object_to_file(order_pivot_test, TEST_PROCESSED)

Пусть сохранения /content/drive/MyDrive/docs/keepForever/mipt/flactory_dataton/df/test_preprocessed_df.pkl


# Бейзлайн модели

## Бейзлайн с val набором и выводом прогноза для test

#### Загрузка готовых наборов, полученных на предыдущем шаге и прочих требуемых данных

In [7]:
order_pivot_train = load_object_from_file(TRAIN_PROCESSED)
order_pivot_val = load_object_from_file(VAL_PROCESSED)
order_pivot_test = load_object_from_file(TEST_PROCESSED)

In [8]:
with open(TRAIN_INPUT_PATH) as file:
    train_file = file.read()

train_json = json.loads(train_file)

with open(VAL_INPUT_PATH) as file:
    val_file = file.read()

val_json = json.loads(val_file)

In [9]:
def get_target_and_feature(orders_df, input_json):
    if 'user' in orders_df.columns:
        print("'user' column is present.")
    else:
        print("'user' column is missing.")

    print(orders_df.index)
    print(orders_df.index.name)

    orders_df.index.name = 'user'

    orders_df.reset_index(inplace=True)

    if 'user' in orders_df.columns:
        print("'user' column is present.")
    else:
        print("'user' column is missing.")

    target_df = pd.DataFrame([(user, data['target']) for user, data in input_json.items()],
                            columns=['user', 'target'])

    final_df = orders_df.merge(target_df, on='user', how='left')
    final_df = final_df.fillna(0)
    print(final_df.isna().any().any())

    user_id = final_df.drop('user', axis=1)
    features = final_df.drop('target', axis=1)
    target = final_df['target']

    label_encoder = LabelEncoder()
    target.info()
    target = label_encoder.fit_transform(target)

    final_df = None
    target_df = None

    non_numeric_columns = features.select_dtypes(exclude=[np.number]).columns

    for col in non_numeric_columns:
        try:
            features[col] = pd.to_numeric(features[col], errors='coerce')
        except Exception as e:
            print(f"Couldn't convert {col}: {e}")

    features = features.fillna(0)
    print(features.isna().any().any())

    len(features.select_dtypes(exclude=[np.number]).columns)

    return features, target

In [10]:
def get_test_features_and_userid(orders_df):
    if 'user' in orders_df.columns:
        print("'user' column is present.")
    else:
        print("'user' column is missing.")

    print(orders_df.index)
    print(orders_df.index.name)

    orders_df.index.name = 'user'

    orders_df.reset_index(inplace=True)

    if 'user' in orders_df.columns:
        print("'user' column is present.")
    else:
        print("'user' column is missing.")

    orders_df = orders_df.fillna(0)
    print(orders_df.isna().any().any())

    user_id = orders_df['user']
    orders_df.drop('user', axis=1)

    non_numeric_columns = orders_df.select_dtypes(exclude=[np.number]).columns

    for col in non_numeric_columns:
        try:
            orders_df[col] = pd.to_numeric(orders_df[col], errors='coerce')
        except Exception as e:
            print(f"Couldn't convert {col}: {e}")

    orders_df = orders_df.fillna(0)
    print(orders_df.isna().any().any())
    print(len(orders_df.select_dtypes(exclude=[np.number]).columns))

    return orders_df, user_id

In [11]:
X_train, y_train = get_target_and_feature(order_pivot_train, train_json)
train_json = None

'user' column is missing.
Index(['user_1', 'user_10', 'user_100', 'user_1000', 'user_100000', 'user_100001', 'user_100002',
       'user_100003', 'user_100004', 'user_100005',
       ...
       'user_99990', 'user_99991', 'user_99992', 'user_99993', 'user_99994', 'user_99995',
       'user_99996', 'user_99997', 'user_99998', 'user_99999'],
      dtype='object', name=0, length=115509)
0
'user' column is present.
False
<class 'pandas.core.series.Series'>
Int64Index: 115509 entries, 0 to 115508
Series name: target
Non-Null Count   Dtype 
--------------   ----- 
115509 non-null  object
dtypes: object(1)
memory usage: 1.8+ MB
False


In [12]:
X_validate, y_validate = get_target_and_feature(order_pivot_val, val_json)
val_json = None

'user' column is missing.
Index(['user_127756', 'user_127757', 'user_127758', 'user_127759', 'user_127761', 'user_127762',
       'user_127763', 'user_127764', 'user_127765', 'user_127766',
       ...
       'user_155193', 'user_155194', 'user_155195', 'user_155196', 'user_155197', 'user_155198',
       'user_155199', 'user_155200', 'user_155201', 'user_155202'],
      dtype='object', name=0, length=24871)
0
'user' column is present.
False
<class 'pandas.core.series.Series'>
Int64Index: 24871 entries, 0 to 24870
Series name: target
Non-Null Count  Dtype 
--------------  ----- 
24871 non-null  object
dtypes: object(1)
memory usage: 388.6+ KB
False


In [13]:
X_test, test_user_id = get_test_features_and_userid(order_pivot_test)
test_json = None

'user' column is missing.
Index(['user_155204', 'user_155206', 'user_155207', 'user_155208', 'user_155209', 'user_155210',
       'user_155211', 'user_155212', 'user_155213', 'user_155214',
       ...
       'user_173977', 'user_173978', 'user_173979', 'user_173980', 'user_173981', 'user_173982',
       'user_173983', 'user_173984', 'user_173985', 'user_173986'],
      dtype='object', name=0, length=16922)
0
'user' column is present.
False
False
0


#### Выравнивание столбцов наборов train, val и test

In [14]:
def align_dataframes(dataframes_map):
    all_columns = set()
    for df in dataframes_map.values():
        all_columns = all_columns.union(set(df.columns))

    all_columns_list = list(all_columns)

    for df_name, df in dataframes_map.items():
        missing_columns = all_columns - set(df.columns)
        for column in missing_columns:
            print(f'Добавляем столбец {column} в набор {df_name}.')
            df[column] = 0

        dataframes_map[df_name] = df.reindex(columns=all_columns_list)

    return dataframes_map

In [15]:
print(X_train.shape)
print(X_validate.shape)
print(X_test.shape)

(115509, 461)
(24871, 460)
(16922, 460)


In [16]:
dataframes_map = {
    "X_test": X_test,
    "X_validate": X_validate,
    "X_train": X_train
}

dataframes_map = align_dataframes(dataframes_map)
X_test = dataframes_map['X_test']
X_validate = dataframes_map['X_validate']
X_train = dataframes_map['X_train']

Добавляем столбец items_site_51 в набор X_test.
Добавляем столбец items_site_51 в набор X_validate.


In [17]:
print(X_train.shape)
print(X_validate.shape)
print(X_test.shape)

(115509, 461)
(24871, 461)
(16922, 461)


#### Сохранить наборы в файловую систему

In [18]:
xgb_model = XGBClassifier(colsample_bytree=0.3, learning_rate=0.1, max_depth=5, alpha=10, n_estimators=1000, random_state=42)

xgb_model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_validate, y_validate)], verbose=True)

xgb_pred_validate = xgb_model.predict(X_validate)
print('XGBoost Classifier Validation accuracy:\n', accuracy_score(y_validate, xgb_pred_validate), '\n')
print('XGBoost Classifier - результаты классификации:\n', classification_report(y_validate, xgb_pred_validate))

xgb_pred_test = xgb_model.predict(X_test)

results_df = pd.DataFrame({
    'userid': test_user_id,
    'prediction': xgb_pred_test
})

results_df.to_csv(TEST_PRED, index=False)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.67925
[1]	validation_0-logloss:0.67140
[2]	validation_0-logloss:0.66179
[3]	validation_0-logloss:0.65399
[4]	validation_0-logloss:0.64554
[5]	validation_0-logloss:0.63817
[6]	validation_0-logloss:0.63306
[7]	validation_0-logloss:0.62873
[8]	validation_0-logloss:0.62323
[9]	validation_0-logloss:0.61820
[10]	validation_0-logloss:0.61386
[11]	validation_0-logloss:0.60931
[12]	validation_0-logloss:0.60606
[13]	validation_0-logloss:0.60257
[14]	validation_0-logloss:0.59955
[15]	validation_0-logloss:0.59613
[16]	validation_0-logloss:0.59360
[17]	validation_0-logloss:0.59150
[18]	validation_0-logloss:0.58921
[19]	validation_0-logloss:0.58687
[20]	validation_0-logloss:0.58452
[21]	validation_0-logloss:0.58257
[22]	validation_0-logloss:0.58034
[23]	validation_0-logloss:0.57887
[24]	validation_0-logloss:0.57709
[25]	validation_0-logloss:0.57529
[26]	validation_0-logloss:0.57396
[27]	validation_0-logloss:0.57208
[28]	validation_0-logloss:0.57072
[29]	validation_0-loglos